### SciTec Coding Problem 
##### Written by: Richard Santana

In [1]:
import pandas as pd
import numpy as np

# Importing csv into dataframe
df = pd.read_csv('SciTec_code_problem_data.csv', delimiter = ',', header = None, names = ['Time','Latitude','Longitude','Altitude'])
df

,Time,Latitude,Longitude,Altitude
0,1.532333e+09,40.33990,127.51010,0.50000
1,1.532333e+09,40.27902,127.51786,21.28817
2,1.532333e+09,40.21822,127.52562,41.99302
3,1.532333e+09,40.15751,127.53337,62.61454
4,1.532333e+09,40.09687,127.54113,83.15275
5,1.532333e+09,40.03631,127.54887,103.60762
6,1.532333e+09,39.97584,127.55662,123.97918
7,1.532333e+09,39.91545,127.56436,144.26741
8,1.532333e+09,39.85513,127.57210,164.47232
9,1.532333e+09,39.79490,127.57984,184.59390


#### Method for converting LLA to ECEF

In [2]:
import math

# LLA to ECEF method
def LLA_to_ECEF(lat, long, alt):
    # Latitude/Longitude from degrees to radians
    lat_rad = math.radians(lat)
    long_rad = math.radians(long)
    # Altitude from km to m
    h = alt * 1000
    
    # Parameters
    sin_lat = math.sin(lat_rad)
    cos_lat = math.cos(lat_rad)
    sin_long = math.sin(long_rad)
    cos_long = math.cos(long_rad)
    
    a = 6378137
    b = 6356752.31424518
    e = math.sqrt((a**2 - b**2) / (a**2))
    N = a / (math.sqrt(1 - ((e**2) * (sin_lat**2))))
    
    # ECEF components X, Y, Z
    X = (N + h) * cos_lat * cos_long
    Y = (N + h) * cos_lat * sin_long
    Z = ((((b**2) / (a**2)) * N) + h) * sin_lat
    return X, Y, Z

#### Creating new ECEF data matrix from LLA data matrix

In [3]:
# Original LLA pandas dataframe to numpy matrix
LLA_Data = df.values

# New ECEF data matrix to hold Time, X, Y, Z, Vx, Vy, Vz entries
rows = len(LLA_Data)
cols = 7
ECEF_Data = np.zeros((rows, cols))

# Copying same time column
ECEF_Data[:, 0] = LLA_Data[:, 0]

# Iterating original LLA data and converting to ECEF
for i in range(0, len(LLA_Data)):
    lat_input = LLA_Data[i, 1]
    long_input = LLA_Data[i, 2]
    alt_input = LLA_Data[i, 3]
    X, Y, Z = LLA_to_ECEF(lat_input, long_input, alt_input)
    # Insert ECEF components into new data matrix
    ECEF_Data[i, 1] = X
    ECEF_Data[i, 2] = Y
    ECEF_Data[i, 3] = Z

#### Computing velocity vectors for ECEF data

In [4]:
# Velocity at first input point is < 0, 0, 0 >
ECEF_Data[0, 4] = 0
ECEF_Data[0, 5] = 0
ECEF_Data[0, 6] = 0

# Now using < Vx, Vy, Vz > =  {dx, dy, dz} / dt 
for i in range(1, len(ECEF_Data)):
    # dt
    dt = ECEF_Data[i, 0] - ECEF_Data[i - 1, 0]
    
    # dX
    dX = ECEF_Data[i, 1] - ECEF_Data[i - 1, 1]
    
    # dY
    dY = ECEF_Data[i, 2] - ECEF_Data[i - 1, 2]
    
    # dZ
    dZ = ECEF_Data[i, 3] - ECEF_Data[i - 1, 3]
    
    # Vx = dX / dt
    Vx = dX / dt
    ECEF_Data[i, 4] = Vx
    
    # Vy = dY / dt
    Vy = dY / dt
    ECEF_Data[i, 5] = Vy
    
    # Vz = dZ / dt
    Vz = dZ / dt
    ECEF_Data[i, 6] = Vz
    
# Creating pandas dataframe for ECEF_Data matrix    
ECEF_df = pd.DataFrame(ECEF_Data, columns = ['Time', 'X', 'Y', 'Z', 'Vx', 'Vy', 'Vz'])
ECEF_df

,Time,X,Y,Z,Vx,Vy,Vz
0,1.532333e+09,-2.964584e+06,3.862111e+06,4.107149e+06,0.000000,0.000000,0.000000
1,1.532333e+09,-2.977429e+06,3.877758e+06,4.115433e+06,-2569.023210,3129.304093,1656.866742
2,1.532333e+09,-2.990250e+06,3.893364e+06,4.123632e+06,-2564.231587,3121.265773,1639.810772
3,1.532333e+09,-3.003046e+06,3.908930e+06,4.131747e+06,-2559.179218,3113.176674,1622.974411
4,1.532333e+09,-3.015819e+06,3.924455e+06,4.139776e+06,-2554.541946,3105.070444,1605.848054
5,1.532333e+09,-3.028566e+06,3.939941e+06,4.147721e+06,-2549.367959,3097.122595,1588.938060
6,1.532333e+09,-3.041288e+06,3.955384e+06,4.155582e+06,-2544.484162,3088.708136,1572.254966
7,1.532333e+09,-3.053985e+06,3.970787e+06,4.163360e+06,-2539.376397,3080.579093,1555.446758
8,1.532333e+09,-3.066657e+06,3.986149e+06,4.171052e+06,-2534.461432,3072.422521,1538.514523
9,1.532333e+09,-3.079304e+06,4.001469e+06,4.178662e+06,-2529.333103,3063.994967,1521.979649


#### Linear interpolation methods

In [5]:
# Times column to find closest points
ECEF_Times = ECEF_Data[:, 0]

def Get_LI_Output(x, x0, y0, x1, y1):
    y = ((y0 * (x1 - x)) + (y1 * (x - x0))) / (x1 - x0)
    return y

def LinearInterpolation(input_time):
    # First check out of bounds input
    min_time = 1532332859.04
    max_time = 1532335359.04

    if (input_time < min_time) or (input_time > max_time):
        print("Input time is out of range for linear interpolation")
    else:
        # Find closest time points to input 
        t_x0 = ECEF_Times[ECEF_Times < input_time].max()
        t_x1 = ECEF_Times[ECEF_Times > input_time].min()
        t_x0_row = ECEF_df.index[ECEF_df.Time == t_x0].values.astype(int)[0]
        t_x1_row = ECEF_df.index[ECEF_df.Time == t_x1].values.astype(int)[0]

        # Find Vx
        Vx_y0 = ECEF_Data[t_x0_row, 4]
        Vx_y1 = ECEF_Data[t_x1_row, 4]
        Vx_y = Get_LI_Output(input_time, t_x0, Vx_y0, t_x1, Vx_y1)


        # Find Vy
        Vy_y0 = ECEF_Data[t_x0_row, 5]
        Vy_y1 = ECEF_Data[t_x1_row, 5]
        Vy_y = Get_LI_Output(input_time, t_x0, Vy_y0, t_x1, Vy_y1)


        # Find Vz
        Vz_y0 = ECEF_Data[t_x0_row, 6]
        Vz_y1 = ECEF_Data[t_x1_row, 6]
        Vz_y = Get_LI_Output(input_time, t_x0, Vz_y0, t_x1, Vz_y1)
        
        print("< Vx, Vy, Vz > = ","< ",Vx_y,",", Vy_y,",", Vz_y, " >")

#### Driver method for ECEF velocity vector

In [6]:
def Get_VelocityVector(input_time):
    try:
        # Input found case
        row_number = ECEF_df.index[ECEF_df.Time == input_time].values.astype(int)[0]
        print("< Vx, Vy, Vz > = ", "< ", ECEF_Data[row_number, 4], ", ", ECEF_Data[row_number, 5],", ", ECEF_Data[row_number, 6], " >")
    except IndexError:
        # Linear interpolation case
        row_number = -1
        LinearInterpolation(input_time)

#### Test input for velocity vector output

In [7]:
first_input = 1532335268
second_input = 1532334000
Get_VelocityVector(first_input)
Get_VelocityVector(second_input)

< Vx, Vy, Vz > =  <  2142.677027638553 , -2686.417561316897 , -1826.0186363785087  >
< Vx, Vy, Vz > =  <  -756.786058361941 , 621.3891005685934 , -985.8460058885319  >
